In [ ]:
from selenium import webdriver as wd
import cx_Oracle
import os
import time
import re
p = re.compile('만족')
p1 = re.compile("2019|2018")
p2 = re.compile("개")
p3 = re.compile("일")



os.putenv("NLS_NANG",".UTF8")
connection = cx_Oracle.connect('hr' , '1234' , 'localhost/xe')
cursor = connection.cursor()
#접속할 주소
url = "https://store.musinsa.com/app/contents/bestranking/?d_cat_cd=&u_cat_cd=&range=nw&price=&ex_soldout=&sale_goods=&new_product_yn=&list_kind=&page=1&display_cnt=80&sex=&popup=&sale_rate=&price1=&price2=&chk_new_product_yn=&chk_sale=&chk_soldout="
#자동으로 웹 동작을 할 수 있다
driver = wd.Chrome(executable_path="chromedriver.exe")
#사이트 로드를 10초동안 기다리고 10초 안에 로드 완료시 다음 라인 실행
driver.implicitly_wait(10)
#url에 접속 하겠따.
driver.get(url)
cloth_container = driver.find_element_by_class_name("list-box")
cloth_list = cloth_container.find_elements_by_class_name("li_box")
total_page = int(driver.find_element_by_class_name("is_goods").text.split(":")[-1].replace("개","").replace(",",""))
if(total_page % 80 == 0):
    total_page = total_page // 80
else:
    total_page = (total_page // 80) +1
for page in range(1,(total_page+1)):
    driver.execute_script("listSwitchPage(document.f1, '{}')".format(page))
    time.sleep(3)
    for i in range(0,80):
        cloth_container = driver.find_element_by_class_name("list-box")
        cloth_list = cloth_container.find_elements_by_class_name("li_box")
        cloth_list[i].find_element_by_class_name("img-block").click()


        print("brand: ",driver.find_element_by_css_selector(".product_article>li>.product_article_contents>strong>a"))
        brand = driver.find_element_by_css_selector(".product_article>li>.product_article_contents>strong>a").text
        print("season: ",driver.find_element_by_class_name("product_article"))
        season_div = driver.find_element_by_class_name("product_info_section")
        season_Ul = season_div.find_element_by_class_name("product_article")
        season_li_list = season_Ul.find_elements_by_tag_name("li")
        season = ""
        total_sell = ""
        print(len(season_li_list))
        for index in range(0,len(season_li_list)):
            result = season_li_list[index].find_element_by_class_name("product_article_contents")
            season_test = result.text
            if p1.search(season_test) != None:
                if(season_test.split("/")[-1].replace(" ","") == "남"):
                    season = season_test.replace("남","").replace(" ","")
            if p2.search(season_test) != None:
                total_sell = season_test.split("개")[0].replace(",","")
           
        sex_container = driver.find_element_by_class_name("txt_gender")
        sex = sex_container.find_element_by_tag_name("span").text
        popularity = int(driver.find_element_by_id("pageview_1m").text.replace(",",""))
        title = driver.find_element_by_class_name("product_title").text
        bucket_container = season_div.find_elements_by_tag_name("li")[2]
        bucket = int(bucket_container.find_elements_by_tag_name("strong")[1].text.replace(",",""))
        like_ = int(driver.find_element_by_class_name("prd_like_cnt").text.replace(",",""))
        review = int(driver.find_element_by_class_name("link_type").text.replace(",","").replace("건",""))
        satisfaction_list = driver.find_elements_by_class_name("product_article_contents")
        for index in satisfaction_list:
            if p.search(index.text) != None:
                satisfaction = index.text.split("/")[1].split("%")[0].replace(" ","")
        
        delivery_date_container = driver.find_element_by_class_name("delivery_info_section")
        dLi_list = delivery_date_container.find_elements_by_tag_name("li")
        delivery_date = ""
        price_container = driver.find_element_by_class_name("price_info_section")
        pLi_list = price_container.find_elements_by_tag_name("li")
        price = ""
        sale = ""
        print(len(dLi_list))
        print(len(pLi_list))
        for li in pLi_list:
            print(li.text)
        try:
            if(len(pLi_list) == 6 or len(pLi_list) == 7):
                price = driver.find_element_by_id("sale_price").text.split("원")[0].replace(",","")
                sale = int(driver.find_element_by_class_name("txt_kor_discount").text.split("%")[0])
            else:
                price = driver.find_element_by_id("goods_price").text.replace("원","").replace(",","")
                sale = ""
        except:
            price = driver.find_element_by_id("goods_price").text.replace("원","").replace(",","")
            sale = ""
        


        category_container = driver.find_element_by_class_name('item_categories')
        category_ = category_container.find_elements_by_tag_name("a")[1].text


        print("num: " , i)
        print("brand: " , brand)
        print("season: " , season)
        print("sex: " , sex)
        print("title: " , title)
        print("popularity: " , popularity)
        print("bucket: " , bucket)
        print("like_: " , like_)
        print("review: " , review)
        print("satisfaction: " , satisfaction)
        print("total_sell: " , total_sell)
        print("delivery_date: " , delivery_date)
        print("price",price)
        print("sale" , sale)
        print("category_",category_)

        print("="*130)
        driver.back()
        time.sleep(3)